In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
import pandas as pd
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 100
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 텍스트 전처리
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from konlpy.tag import Okt 
import MeCab
mecab = MeCab.Tagger()
import re 
from string import punctuation
import requests
import pickle
import ast

# 토픽모델링
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis 
import pyLDAvis.gensim

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


#### 미리 만들어둔 리뷰데이터 불러오기

In [3]:
df = pd.read_csv("D:/review.csv")

In [4]:
df = df[df['DATE'] <= '2021-04-04'] # 4월 이후 리뷰

In [5]:
df.shape

(1856, 4)

#### corpus (말뭉치) 생성

In [6]:
corpus = df['REVIEW']
corpus

0       부산에도 기다리던 지역화폐가 나왔네요 앱도 깔끔하고 카드도 바로 신청하고 잘 사용하...
1                                                     화이팅
2                                      기다렸었는데 앞으로 잘 쓸게요^^
3         동백전으로 여기저기 많이 사용해 볼게요. 부산 경제에도 많은 도움이 되면 좋겠습니다.
4                   부산 동백전 쓰기 편하게 잘 만들어졌네요 부산에서 혜택이 많았겠네요
5                       잘 사용해 볼게요. 동백전으로 혜택 많이 받으면 좋겠습니다.
6                                        가자 글로벌 테스트베드 부산!
7       오~ 부산도 드디어 모바일 지역화폐가 나왔네요!~ 앱설치하고 카드신청도 해봤는데요 ...
8                    드디어 기다리던 동백전앱이 출시됐네요 앞으로 이용 많이 하겠습니다
9          군더더기 없는 ui가 마음에 드는군요. 많은 사업장에서 사용가능하면 더 좋겠습니다.
10                                                    좋아요
11      선택약정선택하면 네트워크장애라고 뜨면서 진행되지 않음...부산시에서 개인정보를 팔아...
12                            많은 이용 바랍니다 부산시민과 함께 성장해 갑시다
13      부산을 대표하는 꽃 이름으로 지은 동백전. 이름이 멋지네요. 부산에서 이용하는 혜택...
14                                           편의점에서도 사용되나요
15                                        설치가 아주 간단하고 쉽네요
16                              군더더기 없이 사용하기 편하고 좋습니다. ^^
17            

#### 텍스트 전처리
- 자음모음만으로 구성된 것 제거
- 특수문자 제거
- 숫자 제거

In [7]:
def message_cleaning(docs):

  
   
    # Series의 object를 str로 변경.
    docs = [str(doc) for doc in docs]
    
    
    # 1. 자음 모음 제거하기
    pattern1 = re.compile("[ㄱ-ㅎ]*[ㅏ-ㅢ]*")
    docs = [pattern1.sub("", doc) for doc in docs]
    # 2. 특수문자 제거
    pattern2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")
    docs = [pattern2.sub("", doc) for doc in docs]
    # 3. 숫자 제거
    pattern3 = re.compile('\d+')
    docs = [pattern3.sub("", doc) for doc in docs]
    
    return docs


<>:13: DeprecationWarning: invalid escape sequence \{
<>:16: DeprecationWarning: invalid escape sequence \d
<>:13: DeprecationWarning: invalid escape sequence \{
<>:16: DeprecationWarning: invalid escape sequence \d
<>:13: DeprecationWarning: invalid escape sequence \{
<>:16: DeprecationWarning: invalid escape sequence \d
<ipython-input-7-5090dc8cd3d9>:13: DeprecationWarning: invalid escape sequence \{
  pattern2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")
<ipython-input-7-5090dc8cd3d9>:16: DeprecationWarning: invalid escape sequence \d
  pattern3 = re.compile('\d+')


#### 명사추출

In [8]:
import re
import MeCab # 형태소 분석기
mecab = MeCab.Tagger()
import requests
import pickle
import ast
## mecab형태소 분석기로 명사 추출하는 함수
def mecab_nouns(text): 
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]] 
    
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        #동사(어근)까지 추출할려면 "VV"까지
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR"or token[1] == "NNS"or token[1] == "NP" or token[1] == "NR"or token[1] == "NNS" or token[1]== "SL" :
            nouns.append(token[0])
    return nouns

In [9]:
## 텍스트 정제
cleaned_corpus = message_cleaning(corpus)
print(cleaned_corpus)

['부산에도 기다리던 지역화폐가 나왔네요 앱도 깔끔하고 카드도 바로 신청하고 잘 사용하겠습니다', '화이팅', '기다렸었는데 앞으로 잘 쓸게요', '동백전으로 여기저기 많이 사용해 볼게요 부산 경제에도 많은 도움이 되면 좋겠습니다', '부산 동백전 쓰기 편하게 잘 만들어졌네요 부산에서 혜택이 많았겠네요', '잘 사용해 볼게요 동백전으로 혜택 많이 받으면 좋겠습니다', '가자 글로벌 테스트베드 부산', '오 부산도 드디어 모바일 지역화폐가 나왔네요 앱설치하고 카드신청도 해봤는데요 복잡하지 않고 간편하게 구성되어 있는거 같아 좋네요 저는 수도권에 올라와 있어서 명절에 내려가면 쓰려고 하는데 카드만 긁으면 된다고 하니 엄청 편할거 같네요  내려가면 부모님도 쓰실 수 있게 알려드릴까 해요 지역페이는 수당도 주고 하는거 같던데 나중에 복지비용도 이걸로 쓸 수 있게 나오면 부모님도 어렵지 않게 쓰실 수 있을거 같아 기대됩니다', '드디어 기다리던 동백전앱이 출시됐네요 앞으로 이용 많이 하겠습니다', '군더더기 없는 ui가 마음에 드는군요 많은 사업장에서 사용가능하면 더 좋겠습니다', '좋아요', '선택약정선택하면 네트워크장애라고 뜨면서 진행되지 않음부산시에서 개인정보를 팔아 장사하는지', '많은 이용 바랍니다 부산시민과 함께 성장해 갑시다', '부산을 대표하는 꽃 이름으로 지은 동백전 이름이 멋지네요 부산에서 이용하는 혜택도 많아 기대가 됩니다 앱도 복잡하지 않고 사용하기 편리해 보여 한번 써 봐야겠네요', '편의점에서도 사용되나요', '설치가 아주 간단하고 쉽네요', '군더더기 없이 사용하기 편하고 좋습니다 ', '어플이 깔끔하니 사용하기 좋아요 자주 사용하게 될듯 하네요', '아니 QR코드 결제가 안되냐 기본인데 하 카드 신청도 잘 안되고 미치겄네', '회원가입이 안 돼요 다음주에 만 세 되는 사람인데요 계속 만 세 이상만 가입이 된다고 떠요 어떡해요', '적립한도 볼 수 있게 잘 쓰고 있는데 충전한도 말고 적립한도도 볼 수 있게 하면 훨씬 좋을 것 같네요', '지웠

#### 불용어 처리 및 한글자 제거

In [10]:

def define_stopwords(path):
    
    SW = set()
    #불용어를 추가하는 방법 1.
    #SW.add("동백전")
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word[:-1])
            
    return SW

from tqdm import tqdm_notebook # 시간 바

# 명사 추출한 것 중 SW에 포함되지 않으면서 한글자 제거
def text_tokenizing(corpus):   
    token_corpus = []
    # tqdm을 사용하여 진행 과정을 보기
    for n in tqdm_notebook(range(len(corpus))):
        token_text = mecab_nouns(corpus[n]) # 위에서 정의한 명사추출 함수 실행
        token_text = [word for word in token_text if word not in SW and len(word) >1]
        token_corpus.append(token_text)
    return token_corpus

SW = define_stopwords("D:/연구알바/stopwords-ko.txt")

#### 사용자사전(분리되면 안되는 단어들을 따로 사전에 정의)
> 동백전
하나은행
부산은행
캐쉬백
캐시백
지역화폐
해결방안
고객센터
교통카드
체크카드
불편함
본인인증
본인 인증
인증번호
코나아이
삼성페이
오프라인
온라인
비번
비밀번호
아이디
전화번호
폰 번호
생년월일
홈페이지
소상공인
지원금
로코
큐알코드 등등

In [11]:
tokenized_text = text_tokenizing(cleaned_corpus)
print(tokenized_text)

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1856 [00:00<?, ?it/s]

[['지역화폐', '카드', '신청', '사용'], ['화이팅'], [], ['여기저기', '사용', '경제', '도움'], ['혜택'], ['사용', '혜택'], ['글로벌', '테스트', '베드'], ['모바일', '지역화폐', '설치', '카드', '신청', '구성', '수도', '명절', '카드', '부모', '지역', '페이', '수당', '나중', '복지', '비용', '이걸로', '부모', '기대'], ['출시', '이용'], ['군더더기', 'ui', '마음', '사업장', '사용', '가능'], [], ['선택', '약정', '선택', '네트워크', '장애', '진행', '개인', '정보', '장사'], ['이용', '시민', '성장'], ['대표', '이름', '이름', '이용', '혜택', '기대', '사용', '편리'], ['편의점', '사용'], ['설치'], ['군더더기', '사용'], ['사용', '사용'], ['QR', '코드', '결제', '기본', '카드', '신청'], ['회원가입', '사람', '가입'], ['적립', '한도', '충전', '적립', '한도'], ['마트', '결제', '확인', 'xx', '상황', '충전', '네트워크', '상태', '확인'], ['오류', '충전', '데이터', '상태', '와이파이', '사용', '정보', '오류'], ['오류', '개선', '오류', '캐시백', '문의', '가게', '취소', '재결', '방법', '오류', '해결방안', '불편'], ['실행', '사용', '정보', '취득', '실패'], ['먹통', '설치'], ['오류', '오류', '사용'], ['발급', '한참', '오류', '충전', '개선', '부탁', '하루', '시도', '충전', '금액', '입력', '자동'], ['오류', '설치', '네트워크', '오류'], ['며칠', '가맹점', '조회'], ['카드', '결제', '지연', '오류', '카드', '사용', '오류', '계산', '사람', '카

#### 문서 단어 행렬

In [12]:
def dic_and_bow(clean_text):
    
    # 데이터를 dictionary 형태로 명사 list 만들기 
    dictionary = corpora.Dictionary(clean_text) 
    
    # 출현빈도가 너무 적은 단어는 제거 
    dictionary.filter_extremes(no_below=5) 
    
    # 명사 형태로 말뭉치 만들기 
    corpus = [dictionary.doc2bow(text) for text in clean_text]
    
    # TF-IDF으로 변환 
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    corpus = corpus_tfidf 
    
    return corpus ,dictionary

corpus ,dictionary = dic_and_bow(tokenized_text)

- TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단합니다. 
- TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것입니다. 
- 즉, the나 a와 같이 불용어의 경우에는 모든 문서에 자주 등장하기 마련이기 때문에 자연스럽게 불용어의 TF-IDF의 값은 다른 단어의 TF-IDF에 비해서 낮아지게 됩니다.
> 모든 문서에서 자주 등장하는 단어의 중요도를 낮게 평가 (?!)

- TF-IDF를 돌린 후 명사 추출 > corpus에서 문장에서 단어의 중요도를 평가해서 추출한 것을 다시 명사로 추출하면 그 중요도가 의미가 없어짐(중요하다는 단어가 동사일 경우 중요함에도 사라지기에)

- 명사 추출 후 TF-IDF를 돌리기 > 모든 문서에서 자주 등장하는 명사가 중요하지 않다고 판단하여 핵심 명사들이 사라지는 경우가 생김

- 단순 빈도로 인한 명사 추출뿐인가..

In [13]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 10

NUM_TOPIC_WORDS = 10

def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")

word_dict = {};

for i in range(NUM_TOPICS):

    words = model.show_topic(i)

    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]

pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,업데이트,사용,카드,접속,실행,편리,사용,캐쉬백,접속,편리
1,캐시백,최고,조아,사용,오류,확인,혜택,사용,충전,사용
2,가맹점,캐시백,유용,실행,감사,사용,결제,에러,불가,만족
3,사용,만족,가맹점,카드,카드,충전,업데이트,충전,카드,적립
4,접속,접속,불편함,오류,결제,오류,유용,접속,경제,카드
5,불편,포인트,사용,서버,부족,이용,감사,불편,지역,충전
6,네트워크,업데이트,충전,편리,조아,와이파이,다운,오류,설치,통신
7,감사,감사,프로,네트워크,사용,필요,편리,포인트,편리,업데이트
8,연결,오류,코드,프로,충전,기대,할인,연결,자체,결제
9,결제,적립,아이폰,Good,신청,가능,강추,도움,금액,가입


In [14]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.026722 -0.060092       1        1  11.400055
4      0.086257  0.030061       2        1  10.812815
5      0.025176 -0.033906       3        1  10.793383
7      0.016545 -0.082240       4        1  10.634821
6     -0.062996  0.023317       5        1  10.354063
0     -0.044948 -0.005026       6        1   9.947975
9      0.000576 -0.025708       7        1   9.251542
3      0.024468 -0.005299       8        1   9.251404
1     -0.136054  0.037918       9        1   8.872855
2      0.064256  0.120976      10        1   8.681086, topic_info=      Term        Freq       Total Category  logprob  loglift
72    업데이트   51.000000   51.000000  Default  30.0000  30.0000
139     최고   29.000000   29.000000  Default  29.0000  29.0000
0       사용  100.000000  100.000000  Default  28.0000  28.0000
23      편리   43.000000   43.000000  Default  27.0000  27.0000
7       혜택   25.000000   25.000000  Default  26.0000  26.0000
70      만족   18.000000   18.000000  Default  25.0000  25.0000
46      실행   35.000000   35.000000  Default  24.0000  24.0000
44     캐시백   35.000000   35.000000  Default  23.0000  23.0000
174     조아   17.000000   17.000000  Default  22.0000  22.0000
64      접속   47.000000   47.000000  Default  21.0000  21.0000
180     유용   18.000000   18.000000  Default  20.0000  20.0000
109    캐쉬백   17.000000   17.000000  Default  19.0000  19.0000
98      에러   12.000000   12.000000  Default  18.0000  18.0000
3       카드   56.000000   56.000000  Default  17.0000  17.0000
68      감사   26.000000   26.000000  Default  16.0000  16.0000
62      서버   10.000000   10.000000  Default  15.0000  15.0000
5       경제   10.000000   10.000000  Default  14.0000  14.0000
197     통신    8.000000    8.000000  Default  13.0000  13.0000
80     포인트   21.000000   21.000000  Default  12.0000  12.0000
55     가맹점   27.000000   27.000000  Default  11.0000  11.0000
106     불가   11.000000   11.000000  Default  10.0000  10.0000
107    불편함    7.000000    7.000000  Default   9.0000   9.0000
226     부족    7.000000    7.000000  Default   8.0000   8.0000
6       도움   14.000000   14.000000  Default   7.0000   7.0000
2     지역화폐    8.000000    8.000000  Default   6.0000   6.0000
128     필요   12.000000   12.000000  Default   5.0000   5.0000
36      오류   47.000000   47.000000  Default   4.0000   4.0000
178   Good    6.000000    6.000000  Default   3.0000   3.0000
29      적립   24.000000   24.000000  Default   2.0000   2.0000
140     할인   12.000000   12.000000  Default   1.0000   1.0000
11      페이    3.809547    5.934753   Topic1  -4.1932   1.7282
5       경제    6.537143   10.795743   Topic1  -3.6533   1.6699
106     불가    6.988270   11.603935   Topic1  -3.5865   1.6644
104     자체    4.510102    9.532313   Topic1  -4.0244   1.4232
142    이벤트    1.504493    3.284910   Topic1  -5.1223   1.3907
79      장난    1.856736    4.060573   Topic1  -4.9119   1.3890
165     심플    1.558350    3.504142   Topic1  -5.0871   1.3612
225     발전    2.021136    4.580389   Topic1  -4.8271   1.3534
170     입금    1.668711    3.855929   Topic1  -5.0187   1.3340
10      지역    5.911044   13.908910   Topic1  -3.7539   1.3158
186     제도    2.054551    5.100679   Topic1  -4.8107   1.2622
238     환급    1.707708    4.318703   Topic1  -4.9956   1.2437
198     일정    1.310370    3.360546   Topic1  -5.2604   1.2298
145     어제    1.847238    4.907182   Topic1  -4.9171   1.1945
133     생각    3.018568    8.100006   Topic1  -4.4260   1.1845
235     지금    1.146168    3.168369   Topic1  -5.3943   1.1548
213    인터넷    1.389915    3.866299   Topic1  -5.2015   1.1485
164     가요    1.796834    5.072905   Topic1  -4.9447   1.1337
2     지역화폐    3.068451    8.952616   Topic1  -4.4096   1.1008
132     소비    1.443982    4.330700   Topic1  -5.1634   1.0732
217     발송    1.022234    3.211700   Topic1  -5.5088   1.0267
9       설치    5.633511   18.020562   Topic1  -3.8020   1.0088
175     개꿀    1.814603 